In [1]:
import requests
import pandas as pd

BASE_URL = "https://www.cheapshark.com/api/1.0/deals"

def fetch_all_deals(store_id="1", page_size=60, max_pages=30):
    """
    Fetch multiple pages of CheapShark deals.

    - store_id="1" means Steam
    - page_size=60 is the maximum number of deals per page
    - max_pages is a safety limit so we don't loop forever
    """
    all_deals = []
    page = 0

    while page < max_pages:
        params = {
            "storeID": store_id,
            "pageSize": page_size,
            "pageNumber": page,
            "sortBy": "Savings",  # sort by biggest discount first
        }
        print(f"Requesting page {page} ...")
        resp = requests.get(BASE_URL, params=params, timeout=30)

        if resp.status_code != 200:
            print(f"  ❌ Status code {resp.status_code}, stopping.")
            break

        deals = resp.json()
        print(f"  Got {len(deals)} deals.")

        if not deals:
            # No more data, we've reached the end
            break

        all_deals.extend(deals)

        # If we got fewer than page_size deals, this is probably the last page
        if len(deals) < page_size:
            break

        page += 1

    print(f"\nTotal deals fetched: {len(all_deals)}")

    df = pd.DataFrame(all_deals)

    # ✅ 把 CheapShark 的 releaseDate（Unix 秒）转成日期和年份
    if "releaseDate" in df.columns:
        # 转成真正的时间
        df["release_date"] = pd.to_datetime(
            df["releaseDate"],
            unit="s",
            utc=True,
            errors="coerce"
        )
        # 取出年份（int）
        df["release_year"] = df["release_date"].dt.year

    return df


df_raw = fetch_all_deals()
df_raw.head()


Requesting page 0 ...


  Got 60 deals.
Requesting page 1 ...


  Got 60 deals.
Requesting page 2 ...


  Got 60 deals.
Requesting page 3 ...


  Got 60 deals.
Requesting page 4 ...


...

  Got 60 deals.
Requesting page 27 ...


  Got 60 deals.
Requesting page 28 ...


  Got 60 deals.
Requesting page 29 ...


  Got 60 deals.

Total deals fetched: 1800


,internalName,title,metacriticLink,dealID,storeID,gameID,salePrice,normalPrice,isOnSale,savings,...,steamRatingText,steamRatingPercent,steamRatingCount,steamAppID,releaseDate,lastChange,dealRating,thumb,release_date,release_year
0,JUSTMOVECLEANCITYMESSYBATTLE,Just Move:Clean City Messy Battle,None,9UU%2Fnqm0txIvzHW6PakYvteV4itHS8sAFgo1MJ%2FjJZ...,1,304604,0.79,15.99,1,95.059412,...,None,0,0,3414900,0,1763850518,1.6,https://shared.fastly.steamstatic.com/store_it...,1970-01-01 00:00:00+00:00,1970
1,JUSTMOVEFALLDUNGEONENDLESSABYSS,Just Move Fall Dungeon Endless Abyss,None,0n%2FnB%2By%2Bk3G785meL6tDZ%2FwDfvF2ee2zFJEsxX...,1,306530,0.79,15.99,1,95.059412,...,None,0,0,3856040,0,1764884537,3.3,https://shared.fastly.steamstatic.com/store_it...,1970-01-01 00:00:00+00:00,1970
2,LIVINGBATTLE,LivingBattle,None,6%2FJawAxHTcqG3WqvYHKvOPkqn116HLHEZljlb4EcV8k%3D,1,312088,0.79,15.99,1,95.059412,...,None,0,0,3353830,0,1765216975,3.9,https://shared.fastly.steamstatic.com/store_it...,1970-01-01 00:00:00+00:00,1970
3,UNCLAIMEDWORLD,Unclaimed World,/game/unclaimed-world/,oOIeSWUAU6bwhCt8eVnsnELwb%2FXff3yYYMvmf%2FezlB...,1,107953,0.84,16.99,1,95.055915,...,Mixed,49,174,284100,1475539200,1765233773,7.9,https://shared.fastly.steamstatic.com/store_it...,2016-10-04 00:00:00+00:00,2016
4,PLAYINGHISTORYVIKINGS,Playing History: Vikings,/game/playing-history-vikings/,0Ju1T5oYH0o0zwd%2BWXC9bCiLT%2B96QHC%2FuAswjg44...,1,133503,0.84,16.99,1,95.055915,...,Very Positive,80,76,368760,1413590400,1764977562,3.4,https://shared.fastly.steamstatic.com/store_it...,2014-10-18 00:00:00+00:00,2014


In [2]:
df_raw.to_csv("cheapshark_steam_deals_with_release_year.csv", index=False)
